# Day 10

## Process input data

In [3]:
import numpy as np

In [4]:
def read_data(filename):
    with open (filename) as fh:
        data = np.asarray([int(line.strip()) for line in fh.readlines()])
        
    built_in_adapter = max(data) + 3
    data = np.append(data, [0, built_in_adapter])
    data.sort()
    return data

In [77]:
joltages = read_data("input.txt")
joltages

array([  0,   1,   2,   3,   4,   7,   8,   9,  12,  15,  16,  17,  18,
        19,  22,  25,  28,  29,  32,  35,  36,  37,  40,  41,  42,  43,
        44,  47,  48,  49,  50,  51,  54,  57,  60,  61,  62,  63,  66,
        69,  70,  71,  74,  75,  76,  77,  80,  81,  82,  83,  84,  87,
        88,  89,  90,  91,  94,  97,  98,  99, 100, 103, 104, 105, 106,
       109, 110, 111, 112, 115, 116, 117, 118, 121, 122, 123, 124, 127,
       128, 131, 132, 133, 134, 135, 138, 141, 142, 143, 144, 147, 148,
       149, 150, 153, 156, 159, 160, 161, 162, 163, 166, 167, 168, 169,
       172])

## Part 1

In [15]:
from numba import cuda

In [67]:
@cuda.jit
def calculate_intervals(joltages, output):
    pos = cuda.grid(1)
    if pos < len(joltages) - 1:
        output[pos] = joltages[pos + 1] - joltages[pos]

In [68]:
threadsperblock = 128
blockspergrid = (len(joltages) - 1 + (threadsperblock - 1)) // threadsperblock

In [69]:
output = np.zeros(len(joltages) - 1)
calculate_intervals[blockspergrid, threadsperblock](joltages, output)
output

array([1., 1., 1., 1., 3., 1., 1., 3., 3., 1., 1., 1., 1., 3., 3., 3., 1.,
       3., 3., 1., 1., 3., 1., 1., 1., 1., 3., 1., 1., 1., 1., 3., 3., 3.,
       1., 1., 1., 3., 3., 1., 1., 3., 1., 1., 1., 3., 1., 1., 1., 1., 3.,
       1., 1., 1., 1., 3., 3., 1., 1., 1., 3., 1., 1., 1., 3., 1., 1., 1.,
       3., 1., 1., 1., 3., 1., 1., 1., 3., 1., 3., 1., 1., 1., 1., 3., 3.,
       1., 1., 1., 3., 1., 1., 1., 3., 3., 3., 1., 1., 1., 1., 3., 1., 1.,
       1., 3.])

In [70]:
[ones] = np.where(output==1)
len(ones)

70

In [71]:
[threes] = np.where(output==3)
len(threes)

34

In [72]:
len(ones) * len(threes)

2380

## Part 2

Because of the linked nature of this calculation it doesn't make sense to parallelise.

Copied from [this solution](https://gist.github.com/Battleman/b5a6e171cc4660e69fd51507f73c9190), but converted to CUDA.

In [37]:
joltages = read_data("input.txt")

In [39]:
linkers = {n: 1 for n in joltages}
for i, n1 in enumerate(joltages):
    for j in (i+2, i+3):
        if j < len(joltages) and joltages[j] - n1 <= 3:
            for n2 in joltages[j:]:
                linkers[n2] += linkers[n1]
    
print(linkers[max(joltages)])

48358655787008
